# 自然言語処理の前処理における一連の流れ

①クリーニング処理

②形態素解析

③正規化

④ストップワード除去

⑤単語のベクトル表現


## ①クリーニング処理

<font color="Crimson">クリーニング処理ではノイズとなるものを除去します。</font>  
ノイズとは、例えばインターネット上から拾ってくる文書を前処理する場合、htmlタグなどが含まれていますが、このようなタスクの結果に悪影響を及ぼすようなものをノイズといいます。

In [ ]:
import urllib.request
from bs4 import BeautifulSoup

url = "https://news.livedoor.com/topics/category/main/"
html = urllib.request.urlopen(url)

soup = BeautifulSoup(html, "lxml")

titles = soup.find_all("h3", class_="articleListTtl")
print(titles)

In [ ]:
# stringで文字列のみ抜き出す。
for title in titles:
    print(title.string)

## ②形態素解析
<font color="Crimson">形態素解析とは文章を形態素に分けることです。  
形態素とは意味を持つ言葉の最小単位を指します。</font>

In [ ]:
!pip install mecab-python3

In [ ]:
!pip install unidic-lite

In [ ]:
!pip install --no-binary :all: mecab-python3

In [ ]:
import MeCab
m = MeCab.Tagger (" ")
# 形態素解析した解析結果をparse()によって取得
sentence = '鬼滅の刃の映画を見に行った。'

text = m.parse(sentence)
print(text)

他にも形態素解析ツールには、janomeやjuman++,cabocha, nagisaなどがあります。  
MeCabは形態素解析ツールの中でも知名度が高く、実行速度が早いため、よく利用されます。

## ③正規化
形態素解析した文字には同じ意味でも異なる表記をしているものが存在します。  
例えば、全角の「クルマ」と半角の「ｸﾙﾏ」は別の単語として分類されてしまいます。  
上記のように、<font color="Crimson">これらを同じ単語として処理させるために正規化を行います。</font>

In [ ]:
!pip install neologdn

In [ ]:
import neologdn

# 半角を全角に統一
full_letter = neologdn.normalize("ｸﾙﾏ")
print(full_letter) # クルマ

# 不要なスペース削除
cut_space = neologdn.normalize("こ　ん　に　ち　は　")
print(cut_space) # こんにちは

# 似た文字の統一
unification = neologdn.normalize("-⁃֊⁻₋−‑˗‒–")
print(unification) # -

# 伸ばし棒を一つにする
shorter = neologdn.normalize("とてもきれいだーーーーーー")
print(shorter) # とてもきれいだー

# 繰り返しの制限
cut_repeat = neologdn.normalize("あああいたたたいなぁ", repeat=1)
print(cut_repeat) # あいたいなぁ

他には数字の置換えがあります。  
<font color="Crimson">数字の置換えは、日時や値段に適用させます。基本的にこれらは特徴量として重要ではないので同じ文字に変換します。</font>  
数字そのものをヒットさせるためには正規表現を利用します。数字(正確には整数)は正規表現を用いると[0-9]または\dと表現できます。  
正規表現を使って置き換えるにはre.subメソッドを使います。  

In [ ]:
import re
t = "12月のクリスマスに100万円のダイヤモンドをプレゼントする"
# re.sub("ヒットさせたいもの(正規表現)","置換する文字列","置換したい文章" )
print(re.sub("[0-9]+","0", t)) 

１つの文書中に1回または2回しかでてこない単語は、その文書の意味を捉える特徴量にはなりにくいです。  
そのため、このような単語はリスト化して取り除くか、まとめて1つの特徴量として扱うべきです。  
こうすることによって<font color="Crimson">精度向上と、学習時間の削減を実現できます。</font>

## ④ストップワード除去
<font color="Crimson">ストップワードは自然言語処理をするにおいて、頻出過ぎて特徴量として成立しないような単語を指します。</font>  
例えば、助詞の「は」は、主語の次につくことが多いです。**そのため頻出過ぎて文章の特徴をつかむ単語にはなり得ないのです。**  
ストップワードの取得方法は<font color="Crimson">大体3種類あり、①辞書、②出現頻度、③レアな単語</font>から取得できます。



### ①辞書からストップワードを取得する方法

In [ ]:
# 辞書を使ったストップワード
import os
def download(path):
    # SlothLibと呼ばれるストップワード辞書を読み込み
    url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    if os.path.exists(path):
        print('File already exists.')
    else:
        print('Downloading...')
        urllib.request.urlretrieve(url, path)
        print("Finish!")

download("stopwords")

In [ ]:
with open("./stopwords", "r", encoding="utf-8") as f:
    lines = f.readlines()

stopword_list = [line.split("\n")[0] for line in lines if line.split("\n")[0]]
# 形態素解析した表層形のリスト
text = ["あそこ","に","は","、", "多く", "の", "りんご", "が", "あり", "ます", "。"]

result = [word for word in text if word not in stopword_list]
print(result)

**この方法は自分で独自の辞書を作成することができますが、時間的コストがかかることが欠点です。**

英語のストップワードは、NLTKという自然言語処理のライブラリをインストールすることで取得することが可能です。

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words("english")
print(stop_words)

### ②出現頻度からストップワードを取得する方法

この方法は①の辞書と元となる考え方は同じで、高頻出な単語は、特徴量としての重要度が低いという考えです。　　

また、単語の希少性も加味することが可能です。①とは違ってコーパスごとに特有のストップワードを定めることが出来ます。


In [ ]:
# 吾輩は猫であるのデータセット  
!wget https://aiacademy.jp/dataset/neko.txt -O neko.txt

MeCabにより出現回数の多い単語を数え、その単語をストップワードにする方法を学びます。
出現回数を数えるにはCounterメソッドを使います。<font color="Crimson">頻度の高いものはmost_common()を使うことで引数に指定した数だけ上位から取り出すことが可能です。</font>

In [ ]:
import MeCab
from collections import Counter
import re

# ファイルの読み込み
with open("neko.txt", "r", encoding="utf-8") as f:
    file = f.readlines()

# 形態素解析した単語を辞書型に書き換える
r = []
for line in file:
    if len(line.split("\t")) == 6:
        if len(line.split("\t")[3].split("-")) > 1:
            r.append({"surface":line.split("\t")[0],
                     "base":line.split("\t")[2],
                     "pos":line.split("\t")[3].split("-")[0],
                     "pos1":line.split("\t")[3].split("-")[1]})
        else:
            r.append({"surface":line.split("\t")[0],
            "base":line.split("\t")[2],
            "pos":line.split("\t")[3].split("-")[0],
            "pos1":""})
surface = []
for dc in r:
    surface.append(dc["surface"])

count = Counter()
for word in surface: #数え上げる
    count[word] += 1
#上位10だけ取り出す
stopwords = [word[0] for word in count.most_common(10)]
print("most_common(10):", count.most_common(10))
print(stopwords)

## ⑤単語のベクトル表現
<font color="Crimson">単語をベクトル化することで単語の類似度を測ることが可能になるなど扱いやすくなります。</font>  

### one-hot表現
one-hot表現はある要素だけが1で、他の要素を0で表現する方法です。この方法を行うことで、単語を固定長のベクトルに変換できます。例えば、この単語ベクトルを入力とすればニューラルネットワークを固定したニューロンの数で構築することが可能です。  

しかしながら、この表現はベクトル間の演算では何も意味のない結果を得ることの方が多いことが欠点です。つまり無駄が多いのです。また、1つの単語につき1次元使うので、単語の数が多くなるにつれて高次元になってしまいます。

### 単語の分散表現  
分散表現とは単語を高次元の実数ベクトルで表現することです。  
<font color="Crimson">one-hot表現とは異なり、要素ごとでベクトル化するのではなく、単語ごとでベクトル化します。</font>  
この時、意味が近いほど距離が近くなるように実数ベクトルが与えられます。(つまり似た単語には似たようなベクトルが与えられるということになります。)  
例えば、「クロワッサン」と「ドーナツ」をハイブリットさせた「クロワッサンドーナツ」というものがあります。この「クロワッサンドーナツ」を表現したい場合は「クロワッサン」と「ドーナツ」の分散表現を組み合わせるだけで表現することが出来ます。
単語の分散表現を獲得する方法の１つに**word2vec**というものがあります。
word2vecを使えば有名な例ですが、
「King」-「Man」＋「Woman」＝「Queen」という出力をさせることが可能です。
word2vecのText8Corpusでコーパス化するファイルresult.txtを用意します。これは先ほど形態素解析したものを辞書型に変えた処理を用いることで簡単に行うことが可能です。

In [ ]:
r = []
for line in file:
    if len(line.split("\t")) == 6:
        if len(line.split("\t")[3].split("-")) > 1:
            r.append({"surface":line.split("\t")[0],
                     "base":line.split("\t")[2],
                     "pos":line.split("\t")[3].split("-")[0],
                     "pos1":line.split("\t")[3].split("-")[1]})
        else:
            r.append({"surface":line.split("\t")[0],
            "base":line.split("\t")[2],
            "pos":line.split("\t")[3].split("-")[0],
            "pos1":""})
for dc in r:
    if dc["surface"] not in stopwords:
        result.append(dc["surface"])

with open("result.txt", "w", encoding="utf-8") as f:
    for word in result:
        f.write(word + "\n")

In [ ]:
from gensim.models import word2vec

data = word2vec.Text8Corpus('result.txt') # 形態素解析して表層形のみ書いてあるファイル

# size:圧縮したい次元数
# min_count:最低出現数
# window:ある単語の前後でみる単語数
# iter:反復数
model = word2vec.Word2Vec(data, size=100, min_count=5, window=5, iter=100)
# word2vecで表現された猫の単語ベクトル
print(model["猫"])

# 形態素解析とは
形態素解析（morphological analysis）とは、検索エンジンにも用いられている自然言語処理の手法の一つで、ある文章・フレーズを「意味を持つ最小限の単位（＝単語）」に分解して、文章やフレーズの内容を判断するために用いられます。
例えば、以下のような文章があった場合それを形態素解析してみます。

##MeCabとJanome
Pythonで形態素解析を行うにはMeCabとJanomeというのがあります。
MeCabは形態素解析ツールの中でも知名度が高く、実行速度が早いという特徴があります。
しかし、インストールが環境によって少々時間がかかってしまいます。
Janomeはインストールが比較的簡単に出来ますが、MeCabに比べ速度が遅かったりします。

### Janomeのインストール  
http://mocobeta.github.io/janome/

In [ ]:
!pip install janome

##Janomeで形態素解析  
形態素解析をするには、tokenize()を利用します。

In [ ]:
from janome.tokenizer import Tokenizer
t = Tokenizer()
for token in t.tokenize('すもももももももものうち'):
    print(token)

### ユーザー定義辞書の作成方法
単語を独自に追加することも可能です。  
その場合、作成したユーザー定義辞書をToknizer()の引数に指定します。  
辞書ファイルはカンマ区切りで作成します。  
CSVデータのフォーマットですが、左から次のようになります。

In [ ]:
#表層形,左文脈ID,右文脈ID,コスト,品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音

となります。
このとき、「ヘッダーなし」かつ「CSV形式」で作成する事に注意です。  
左文脈IDや右文脈IDなどの詳細はこちらを参考にしてみてください。  
http://taku910.github.io/mecab/dic.html  

＊表層形とは、活用や表記揺れなどを考慮した、本文中において、文字列として実際に出現する形式のことです。  
※これ以降に作成する全てのcsvファイルの保存先は、これから作成及び実行するPythonプログラムと同じ場所に配置してください。  
【参考リンク】  
usedic.csv内に独自の単語を定義しています。  
https://mocobeta.github.io/janome/


In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/AI Academy

In [ ]:
#pandasでの辞書の作成
import pandas as pd

note = [["東京スカイツリー",1288,1288,4569,"名詞","固有名詞","一般","*","*","*","東京スカイツリー","トウキョウスカイツリー","トウキョウスカイツリー"],
        ["東武スカイツリーライン",1288,1288,4700,"名詞","固有名詞","一般","*","*","*","東武スカイツリーライン","トウブスカイツリーライン","トウブスカイツリーライン"],
        ["とうきょうスカイツリー駅",1288,1288,4143,"名詞","固有名詞","一般","*","*","*","とうきょうスカイツリー駅","トウキョウスカイツリーエキ","トウキョウスカイツリーエキ"]]

df2 = pd.DataFrame(note)
df2

In [ ]:
#csvファイルとして保存
df2.to_csv("sample.csv", index=False, header=None)

In [ ]:
#辞書を使った形態素解析
from janome.tokenizer import Tokenizer
t = Tokenizer("sample.csv", udic_enc="utf8")
for token in t.tokenize(u'東京スカイツリーへのお越しは、東武スカイツリーライン「とうきょうスカイツリー駅」が便利です。'):
    print(token)

Tokenizerの第一引数に辞書を指定することで、  
形態素解析する際に、固有名詞が細かく分解（形態素）されるの防ぎます。

### 情報を抽出する
ここでは、janomeを用いて、品詞や表層形、活用形等の情報をそれぞれ取り出してみます。  
表層形とは、活用や表記揺れなどを考慮した、本文中において、文字列として実際に出現する形式のことです。

In [ ]:
from janome.tokenizer import Tokenizer

t = Tokenizer()
tokens = t.tokenize('すもももももももものうち')

for token in tokens:
     print(token)

"""
すもも   名詞,一般,*,*,*,*,すもも,スモモ,スモモ
も 助詞,係助詞,*,*,*,*,も,モ,モ
もも  名詞,一般,*,*,*,*,もも,モモ,モモ
も 助詞,係助詞,*,*,*,*,も,モ,モ
もも  名詞,一般,*,*,*,*,もも,モモ,モモ
の 助詞,連体化,*,*,*,*,の,ノ,ノ
うち  名詞,非自立,副詞可能,*,*,*,うち,ウチ,ウチ
"""

for token in tokens:
    print(token.surface, '\t')                  # 表層形
    print(token.part_of_speech.split(',')[0] )  # 品詞
    print(token.part_of_speech.split(',')[1] )  # 品詞細分類1
    print(token.part_of_speech.split(',')[2] )  # 品詞細分類2
    print(token.part_of_speech.split(',')[3] )  # 品詞細分類3
    print(token.infl_type)                      # 活用型
    print(token.infl_form)                      # 活用形
    print(token.base_form)                      # 原形
    print(token.reading)                        # 読み
    print(token.phonetic)                       # 発音
    print(token.node_type)                      # node_type

###品詞判定
次に品詞判定を行います。
品詞の抽出には、part_of_speech.split()を利用し、if文を用いて判定を行います。


In [ ]:
from janome.tokenizer import Tokenizer

t = Tokenizer()
tokens = t.tokenize('すもももももももものうち')
for token in tokens:
    # 品詞を取り出す。
    part_of_speech = token.part_of_speech.split(',')[0]

    if part_of_speech == '名詞':
        print(token.surface)

### 読み込んだデータをリスト化する

In [ ]:
from janome.tokenizer import Tokenizer
# from gensim import corpora, matutils
# from gensim import corpora
import pandas as pd

t = Tokenizer()
data = list(pd.read_csv('sample.csv')['東京スカイツリー'].values)

In [ ]:
data

### 名詞と動詞と助動詞のみを抽出する

In [ ]:
all_dictionary_norm, all_dictionary_verb , all_dictionary_averb = [], [], []

for i in range(len(data)):
    dictionary_norm, dictionary_verb, dictionary_averb= [], [], []

    try:
        tokendata = t.tokenize(data[i])
    except:
        continue

    for token in tokendata:
        base, part = token.base_form, token.part_of_speech

        if '動詞' in part:
            dictionary_verb += [base]
        elif '名詞' in part:
            dictionary_norm += [base]
        elif '助動詞' in part:
            dictionary_norm += [base]

    all_dictionary_norm += [dictionary_norm]
    all_dictionary_verb += [dictionary_verb]
    all_dictionary_averb+= [dictionary_averb]

print("名詞: ", all_dictionary_norm )
print("動詞: ", all_dictionary_verb)
print("助動詞:", all_dictionary_averb)

### csvを形態素解析

In [ ]:
note1 =[["トマト 袋 スタンドポリ","398円","429円"],
        ["ミニトマト 1個","158円","170円"],
        ["トマト袋 無選別 500g","398円","429円"]]

In [ ]:
note2 =[["トマト","カスタム名詞","トマト"],
        ["ルネッサンストマト","カスタム名詞","ルネッサンストマト"],
        ["ミニトマト","カスタム名詞","ミニトマト"]]

In [ ]:
df3 = pd.DataFrame(note1)
display(df3)
df4 = pd.DataFrame(note2)
display(df4)

In [ ]:
df3.to_csv("food.csv", index=False, header=None)
df4.to_csv("userdic2.csv", index=False, header=None)

In [ ]:
import csv
from janome.tokenizer import Tokenizer
t = Tokenizer("userdic2.csv", udic_type="simpledic", udic_enc="utf8")

with open('food.csv', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)
    for columns in reader:
        for i in t.tokenize(columns[0]):
            print(i)
        print()

### 文字をカウントする
今回は、著作権の切れた小説文のデータを青空文庫からデータを使って文字をカウントしていきます。  
http://www.aozora.gr.jp/index_pages/person_a.html

In [ ]:
# zipファイルのnameを確認する
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/AI Academy/52409_ruby_51058.zip') as zf:
    for info in zf.infolist():
        print(info.filename)

In [ ]:
from janome.tokenizer import Tokenizer
import os.path
import urllib.request as request

target_url = "http://www.aozora.gr.jp/cards/000879/files/52409_ruby_51058.zip"

download_zip = "52409_ruby_51058.zip" # ダウンロードしたzip

if not os.path.exists(download_zip):
    print("downloading...")
    request.urlretrieve(target_url, download_zip)
    print("done!")

zip_file = zipfile.ZipFile(download_zip, "r")
f = zip_file.open("01jo.txt", "r") # withを使ってopenでも良いです。
data = f.read()
f.close()

text_data = data.decode("shift_jis") # 文字化け対策

# 形態素解析処理
t = Tokenizer()

word_dic = {}
lines = text_data.split("\r\n") # 改行コードの分割

for line in lines:
    li = t.tokenize(line)
    for w in li:
        word = w.surface
        part = w.part_of_speech

        if part.find("名詞") < 0: # 漢字の変換ミスに注意
            continue
        if not word in word_dic:
            word_dic[word] = 0
        word_dic[word] += 1

keys = sorted(word_dic.items(), key = lambda x:x[1], reverse=True)
for word, cnt in keys[:100]:
    print(word, cnt)

# 係り受け解析入門

係り受け解析とは文節間の修飾する（係る）、修飾される（受ける）の関係を調べる事です。